In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def entropy_weight(data):
    """
    计算数据集中每个特征的熵权。

    Args:
        data (DataFrame): 输入的Pandas DataFrame，其中包含数值型数据用于计算熵权。

    Returns:
        Series: 返回一个Pandas Series，包含每个特征的熵权，权重和为1。
    """

    # 数据标准化
    # 使用MinMaxScaler将数据标准化到0和1之间，确保正值且符合熵的计算要求
    scaler = MinMaxScaler()
    normalized_data = pd.DataFrame(
        scaler.fit_transform(data), columns=data.columns)

    # 避免对数运算中的0值
    # 对所有数值加上一个非常小的数(1e-12)，防止对数函数中出现对数底为零的情况
    normalized_data += 1e-12

    # 计算每个特征的熵
    # 计算归一化数据的比例
    proportions = normalized_data.div(normalized_data.sum(axis=0), axis=1)
    # 计算信息熵
    entropy = -np.sum(proportions * np.log(proportions), axis=0)

    # 计算熵权
    # 计算归一化常数k
    k = 1 / np.log(len(normalized_data))
    # 使用1减去每个特征的熵，并通过总熵进行归一化计算得到熵权
    entropy_weight = (1 - entropy) / (len(data.columns) - entropy.sum())

    # 归一化熵权，使得所有权重加和为1
    return entropy_weight / entropy_weight.sum()

In [2]:
# 读取数据
survey = pd.read_csv('./input/survey_data.csv', encoding='utf-8')
survey.describe(include='all')

,性别,考研租房意向,第几次考研,租房倾向,与学校距离,交通便利程度,附近商场,附近医院,附近便利店,是否旁边需要地铁,...,空调是否必要,热水器是否必要,暖气是否必要,床是否必要,网络是否必要,租金价格重视度,押金价格重视度,房源真实程度重视度,预计租房时长,租房预算
count,242,242,242,242,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,...,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242,242
unique,2,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,7
top,男,有意向,1,整租,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6～12个月,1000～1500
freq,148,151,115,182,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,199
mean,NaN,NaN,NaN,NaN,4.256198,4.880165,1.892562,4.450413,4.561983,4.276860,...,4.714876,4.330579,4.644628,4.570248,4.334711,3.950413,3.979339,3.917355,NaN,NaN
std,NaN,NaN,NaN,NaN,0.778646,0.372971,0.310311,0.750947,0.838815,0.902855,...,0.559069,0.891532,0.828532,0.802759,0.959531,0.909619,0.890001,0.934243,NaN,NaN
min,NaN,NaN,NaN,NaN,3.000000,3.000000,1.000000,1.000000,1.000000,2.000000,...,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,4.000000,5.000000,2.000000,4.000000,4.000000,4.000000,...,5.000000,4.000000,5.000000,4.000000,4.000000,3.000000,3.000000,3.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,4.000000,5.000000,2.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,5.000000,5.000000,2.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,NaN,NaN


In [3]:
# 选择相关的列
from tabulate import tabulate
features = ['附近商场', '附近医院', '附近便利店', '是否旁边需要地铁', '是否旁边需要公交车',
            '房间面积重视度', '房间朝向重视度', '空调是否必要', '热水器是否必要',
            '暖气是否必要', '网络是否必要', '租房预算数值']

# 映射租房预算
budget_mapping = {'500以下': 250,
                  '500～1000': 750,
                  '1000～1500': 1250,
                  '1500～2000': 1750,
                  '2000～3000': 2500,
                  '3000～5000': 4000,
                  '5000以上': 6000}
survey['租房预算数值'] = survey['租房预算'].map(budget_mapping)

# 提取特征数据
feature_data = survey[features]

# 最大最小标准化
scaler = MinMaxScaler()
feature_data_normalized = pd.DataFrame(
    scaler.fit_transform(feature_data), columns=features)

feature_scores_squared = feature_data_normalized**2
weights_I = feature_scores_squared.sum() / feature_scores_squared.sum().sum()

print(tabulate(weights_I.reset_index(), headers='keys',
      tablefmt='psql', showindex=False))

+--------------------+------------+
| index              |          0 |
|--------------------+------------|
| 附近商场           | 0.103135   |
| 附近医院           | 0.090034   |
| 附近便利店         | 0.0966888  |
| 是否旁边需要地铁   | 0.0769797  |
| 是否旁边需要公交车 | 0.078306   |
| 房间面积重视度     | 0.0643398  |
| 房间朝向重视度     | 0.113639   |
| 空调是否必要       | 0.0986252  |
| 热水器是否必要     | 0.0858262  |
| 暖气是否必要       | 0.100867   |
| 网络是否必要       | 0.0869304  |
| 租房预算数值       | 0.00462944 |
+--------------------+------------+


In [4]:
rental = pd.read_csv('./input/filtered_data.csv', encoding='utf-8')
survey.describe(include='all')

,性别,考研租房意向,第几次考研,租房倾向,与学校距离,交通便利程度,附近商场,附近医院,附近便利店,是否旁边需要地铁,...,热水器是否必要,暖气是否必要,床是否必要,网络是否必要,租金价格重视度,押金价格重视度,房源真实程度重视度,预计租房时长,租房预算,租房预算数值
count,242,242,242,242,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,...,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242,242,242.000000
unique,2,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,7,NaN
top,男,有意向,1,整租,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6～12个月,1000～1500,NaN
freq,148,151,115,182,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,199,NaN
mean,NaN,NaN,NaN,NaN,4.256198,4.880165,1.892562,4.450413,4.561983,4.276860,...,4.330579,4.644628,4.570248,4.334711,3.950413,3.979339,3.917355,NaN,NaN,1300.619835
std,NaN,NaN,NaN,NaN,0.778646,0.372971,0.310311,0.750947,0.838815,0.902855,...,0.891532,0.828532,0.802759,0.959531,0.909619,0.890001,0.934243,NaN,NaN,470.906282
min,NaN,NaN,NaN,NaN,3.000000,3.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,250.000000
25%,NaN,NaN,NaN,NaN,4.000000,5.000000,2.000000,4.000000,4.000000,4.000000,...,4.000000,5.000000,4.000000,4.000000,3.000000,3.000000,3.000000,NaN,NaN,1250.000000
50%,NaN,NaN,NaN,NaN,4.000000,5.000000,2.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,NaN,NaN,1250.000000
75%,NaN,NaN,NaN,NaN,5.000000,5.000000,2.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,NaN,NaN,1250.000000


In [5]:
# 定义需要进行独热编码的列
columns_to_encode = ['出租方式', '朝向']

# 只对指定的列进行独热编码
data_encoded = pd.get_dummies(rental, columns=columns_to_encode)

In [6]:
# 计算朝向类别的权重
direction_columns = [
    col for col in data_encoded.columns if col.startswith('朝向_')]
direction_weights = entropy_weight(data_encoded[direction_columns])

# 计算朝向特征的整体权重
overall_direction_weight = direction_weights.mean()

# 定义除“朝向”外的其他特征
other_columns = ['租金', '面积', '地铁步行距离', '公交步行距离', '便利店', '商场',
                 '附近医院数', '空调', '热水器', '暖气', '网络']

# 计算其他特征的权重
other_weights = entropy_weight(data_encoded[other_columns])

# 将“朝向”的整体权重添加到权重列表中
weights_II = pd.concat([other_weights, pd.Series(
    [overall_direction_weight], index=['朝向'])])

print(tabulate(weights_II.reset_index(),
      headers='keys', tablefmt='psql', showindex=False))

+--------------+-----------+
| index        |         0 |
|--------------+-----------|
| 租金         | 0.0970101 |
| 面积         | 0.0980734 |
| 地铁步行距离 | 0.0965481 |
| 公交步行距离 | 0.0979466 |
| 便利店       | 0.0978079 |
| 商场         | 0.0976184 |
| 附近医院数   | 0.0981307 |
| 空调         | 0.0843682 |
| 热水器       | 0.0836726 |
| 暖气         | 0.0819406 |
| 网络         | 0.0668834 |
| 朝向         | 0.0666667 |
+--------------+-----------+


In [7]:
# 特征名映射
feature_mapping = {
    '租房预算数值': '租金',
    '房间面积重视度': '面积',
    '附近便利店': '便利店',
    '是否旁边需要地铁': '地铁步行距离',
    '是否旁边需要公交车': '公交步行距离',
    '附近商场': '商场',
    '附近医院': '附近医院数',
    '空调是否必要': '空调',
    '热水器是否必要': '热水器',
    '暖气是否必要': '暖气',
    '网络是否必要': '网络',
    '房间朝向重视度': '朝向'
}

weights_I_adjusted = weights_I.rename(index=feature_mapping)

combined_weights = pd.concat([weights_I_adjusted, weights_II], axis=1)

# 计算加权平均
combined_weights['Final_Weight'] = (
    combined_weights.iloc[:, 0] * 3 + combined_weights.iloc[:, 1]) / 4

# 输出最终权重
print(tabulate(combined_weights['Final_Weight'].reset_index(
), headers='keys', tablefmt='psql', showindex=False))

+--------------+----------------+
| index        |   Final_Weight |
|--------------+----------------|
| 商场         |      0.101756  |
| 附近医院数   |      0.0920581 |
| 便利店       |      0.0969686 |
| 地铁步行距离 |      0.0818718 |
| 公交步行距离 |      0.0832161 |
| 面积         |      0.0727732 |
| 朝向         |      0.101896  |
| 空调         |      0.0950609 |
| 热水器       |      0.0852878 |
| 暖气         |      0.0961352 |
| 网络         |      0.0819186 |
| 租金         |      0.0277246 |
+--------------+----------------+


In [8]:
# 确定评价矩阵的列，即所有计算权重的指标
evaluation_columns = ['租金', '面积', '地铁步行距离',
                      '公交步行距离', '便利店', '商场',
                      '附近医院数', '空调', '热水器',
                      '暖气', '网络', '朝向']
rental['朝向'] = rental['朝向'].str.contains('南').astype(int)
evaluation_matrix = rental[evaluation_columns]
# print(evaluation_matrix.describe(include='all'))

In [9]:
# def evaluate_rent(x):
#     """
#     评估租金的隶属度分布。

#     Args:
#         x (float): 租金的数值。

#     Returns:
#         list: 返回一个五元素列表，代表租金从极低到极高的隶属度分布。
#     """
#     if x < 1500:
#         return [0.5, 0.3, 0.1, 0.1, 0]
#     elif x < 2000:
#         return [0.3, 0.4, 0.2, 0.1, 0]
#     elif x < 2600:
#         return [0.1, 0.3, 0.4, 0.2, 0]
#     elif x < 4000:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     else:
#         return [0, 0, 0.1, 0.3, 0.6]


# def evaluate_area(x):
#     """
#     评估房间面积的隶属度分布。

#     Args:
#         x (float): 房间面积的数值。

#     Returns:
#         list: 返回一个五元素列表，代表面积从极小到极大的隶属度分布。
#     """
#     if x < 45:
#         return [0.5, 0.3, 0.1, 0.1, 0]
#     elif x < 60:
#         return [0.3, 0.4, 0.2, 0.1, 0]
#     elif x < 87:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x < 120:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     else:
#         return [0, 0, 0.1, 0.2, 0.7]


# def evaluate_subway(x):
#     """
#     评估地铁距离的隶属度分布。

#     Args:
#         x (float): 地铁距离的数值。

#     Returns:
#         list: 返回一个五元素列表，代表地铁距离从极近到极远的隶属度分布。
#     """
#     if x < 300:
#         return [0, 0, 0.1, 0.2, 0.7]
#     elif x < 500:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     elif x < 800:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x < 1100:
#         return [0.2, 0.3, 0.3, 0.1, 0.1]
#     else:
#         return [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_bus(x):
#     """
#     评估公交距离的隶属度分布。

#     Args:
#         x (float): 公交距离的数值。

#     Returns:
#         list: 返回一个五元素列表，代表公交距离从极近到极远的隶属度分布。
#     """
#     if x < 200:
#         return [0, 0, 0.1, 0.2, 0.7]
#     elif x < 300:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     elif x < 400:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x < 500:
#         return [0.2, 0.3, 0.3, 0.1, 0.1]
#     else:
#         return [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_convenience(x):
#     """
#     根据便利设施的数量评估其隶属度分布。

#     Args:
#         x (int): 便利设施的数量。

#     Returns:
#         list: 返回一个五元素列表，代表便利设施数量从少到多的隶属度分布。
#     """
#     if x >= 5:
#         return [0, 0, 0.1, 0.2, 0.7]
#     elif x == 4:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     elif x == 3:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x == 2:
#         return [0.2, 0.3, 0.3, 0.1, 0.1]
#     else:
#         return [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_mall(x):
#     """
#     根据商场的数量评估其隶属度分布。

#     Args:
#         x (int): 商场的数量。

#     Returns:
#         list: 返回一个五元素列表，代表商场数量从少到多的隶属度分布。
#     """
#     if x >= 20:
#         return [0, 0, 0.1, 0.2, 0.7]
#     elif x >= 15:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     elif x >= 10:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x >= 5:
#         return [0.2, 0.3, 0.3, 0.1, 0.1]
#     else:
#         return [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_hospital(x):
#     """
#     根据医院的数量评估其隶属度分布。

#     Args:
#         x (int): 医院的数量。

#     Returns:
#         list: 返回一个五元素列表，代表医院数量从少到多的隶属度分布。
#     """
#     if x >= 5:
#         return [0, 0, 0.1, 0.2, 0.7]
#     elif x == 4:
#         return [0, 0.1, 0.3, 0.4, 0.2]
#     elif x == 3:
#         return [0.1, 0.2, 0.4, 0.2, 0.1]
#     elif x == 2:
#         return [0.2, 0.3, 0.3, 0.1, 0.1]
#     else:
#         return [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_bool_feature(x):
#     """
#     评估布尔型特征的隶属度分布。

#     Args:
#         x (bool): 布尔型特征值。

#     Returns:
#         list: 返回一个五元素列表，代表特征的隶属度分布。
#     """
#     return [0, 0.3, 0.1, 0.1, 0.5] if x else [0.5, 0.3, 0.1, 0.1, 0]


# def evaluate_orientation(x):
#     """
#     根据房间朝向评估其隶属度分布。

#     Args:
#         x (int): 房间朝向值，1 表示朝南。

#     Returns:
#         list: 返回一个五元素列表，代表房间朝向从不合适到合适的隶属度分布。
#     """
#     return [0, 0.1, 0.1, 0.3, 0.5] if x == 1 else [0.5, 0.3, 0.1, 0.1, 0]

In [10]:
def evaluate_rent(x):
    """
    评估租金的隶属度分布。

    Args:
        x (float): 租金的数值。

    Returns:
        list: 返回一个五元素列表，代表租金从极低到极高的隶属度分布。
    """
    centers = [1500, 2000, 2600, 4000, 6000]
    widths = [500, 400, 1400, 2000, 2000]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_area(x):
    """
    评估房间面积的隶属度分布。

    Args:
        x (float): 房间面积的数值。

    Returns:
        list: 返回一个五元素列表，代表面积从极小到极大的隶属度分布。
    """
    centers = [45, 60, 87, 120, 160]
    widths = [22, 30, 33, 40, 40]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_subway(x):
    """
    评估地铁距离的隶属度分布。

    Args:
        x (float): 地铁距离的数值。

    Returns:
        list: 返回一个五元素列表，代表地铁距离从极近到极远的隶属度分布。
    """
    centers = [300, 500, 800, 1100, 1500]
    widths = [150, 200, 300, 300, 400]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_bus(x):
    """
    评估公交距离的隶属度分布。

    Args:
        x (float): 公交距离的数值。

    Returns:
        list: 返回一个五元素列表，代表公交距离从极近到极远的隶属度分布。
    """
    centers = [200, 300, 400, 500, 600]
    widths = [100, 100, 100, 100, 100]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_convenience(x):
    """
    根据便利设施的数量评估其隶属度分布。

    Args:
        x (int): 便利设施的数量。

    Returns:
        list: 返回一个五元素列表，代表便利设施数量从少到多的隶属度分布。
    """
    centers = [0, 1, 2, 3, 4]
    widths = [1, 1, 1, 1, 1]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_mall(x):
    """
    根据商场的数量评估其隶属度分布。

    Args:
        x (int): 商场的数量。

    Returns:
        list: 返回一个五元素列表，代表商场数量从少到多的隶属度分布。
    """
    centers = [0, 5, 10, 15, 20]
    widths = [5, 5, 5, 5, 5]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_hospital(x):
    """
    根据医院的数量评估其隶属度分布。

    Args:
        x (int): 医院的数量。

    Returns:
        list: 返回一个五元素列表，代表医院数量从少到多的隶属度分布。
    """
    centers = [0, 2, 3, 4, 5]
    widths = [1, 1, 1, 1, 1]
    memberships = np.exp(-0.5 * ((x - np.array(centers))
                         ** 2 / np.array(widths) ** 2))
    return memberships.tolist()


def evaluate_bool_feature(x):
    """
    评估布尔型特征的隶属度分布。

    Args:
        x (bool): 布尔型特征值。

    Returns:
        list: 返回一个五元素列表，代表特征的隶属度分布。
    """
    return [0, 0, 0, 0, 1] if x else [1, 0, 0, 0, 0]


def evaluate_orientation(x):
    """
    根据房间朝向评估其隶属度分布。

    Args:
        x (int): 房间朝向值，1 表示朝南。

    Returns:
        list: 返回一个五元素列表，代表房间朝向从不合适到合适的隶属度分布。
    """
    return [0, 0, 0, 0, 1] if x == 1 else [1, 0, 0, 0, 0]

In [11]:
# 应用评价规则
rental['租金评价'] = rental['租金'].apply(evaluate_rent)
rental['面积评价'] = rental['面积'].apply(evaluate_area)
rental['地铁评价'] = rental['地铁步行距离'].apply(evaluate_subway)
rental['公交评价'] = rental['公交步行距离'].apply(evaluate_bus)
rental['便利店评价'] = rental['便利店'].apply(evaluate_convenience)
rental['商场评价'] = rental['商场'].apply(evaluate_mall)
rental['医院评价'] = rental['附近医院数'].apply(evaluate_hospital)
rental['空调评价'] = rental['空调'].apply(evaluate_bool_feature)
rental['热水器评价'] = rental['热水器'].apply(evaluate_bool_feature)
rental['暖气评价'] = rental['暖气'].apply(evaluate_bool_feature)
rental['网络评价'] = rental['网络'].apply(evaluate_bool_feature)
rental['朝向评价'] = rental['朝向'].apply(evaluate_orientation)
evaluation_columns = ['租金评价', '面积评价', '地铁评价', '公交评价', '便利店评价',
                      '商场评价', '医院评价', '空调评价', '热水器评价', '暖气评价', '网络评价', '朝向评价']

In [12]:
# 将评价数据从 DataFrame 转换成 NumPy 数组形式
evaluation_matrix = np.array(rental[evaluation_columns].values.tolist())

# 确保评价矩阵的形状正确
print("评价矩阵形状：", evaluation_matrix.shape)

评价矩阵形状： (18935, 12, 5)


In [13]:
# 评价指标的顺序
evaluation_columns = ['租金', '面积', '地铁步行距离', '公交步行距离', '便利店',
                      '商场', '附近医院数', '空调', '热水器', '暖气', '网络', '朝向']

weights_array = combined_weights['Final_Weight'].loc[evaluation_columns].values

# 输出以检查权重数组
print("重新排序的权重数组：", weights_array)

重新排序的权重数组： [0.02772461 0.0727732  0.08187177 0.08321614 0.09696856 0.10175562
 0.09205815 0.09506095 0.08528782 0.09613518 0.08191863 0.10189604]


In [14]:
# 将权重数组重塑为适合广播的形状
weights_reshaped = weights_array.reshape(1, 12, 1)

# 应用权重并计算加权得分
weighted_scores = np.sum(evaluation_matrix * weights_reshaped, axis=1)

# 定义评价等级
grades = ['差', '较差', '中等', '较好', '优秀']

# 确定每个房源的最终评价等级
final_grades = [grades[np.argmax(score)] for score in weighted_scores]

# 将最终评价结果添加到原始 DataFrame 中
rental['最终评价'] = final_grades
rental[["房源编号"]+['最终评价'] +
       evaluation_columns].to_csv('./output/rental_evaluation.csv', index=False)

In [15]:
from pyecharts.charts import Bar
from pyecharts import options as opts
final_grades_counts = rental['最终评价'].value_counts()
bar = Bar()
bar.add_xaxis(final_grades_counts.index.tolist())
bar.add_yaxis("评价数量", final_grades_counts.values.tolist())

# 设置全局选项
bar.set_global_opts(title_opts=opts.TitleOpts(title="评价等级数量分布"))

# 渲染图表到 HTML 文件中
bar.render("./output/评估成绩分布.html")
bar.render_notebook()

In [16]:
from pyecharts.charts import Pie

final_grades_counts = {
    "优秀": 4218,
    "中等": 217,
    "较差": 67,
    "较好": 48,
    "差": 14385
}

# 创建饼图对象
pie = Pie()
pie.add("评价等级", [list(z) for z in final_grades_counts.items()])


pie.set_global_opts(title_opts=opts.TitleOpts(title="评价等级分布"))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)"))

pie.render("./output/评估成绩分布饼图.html")
pie.render_notebook()